In [ ]:
import pandas as pd

#task 1
df = pd.io.gbq.read_gbq('''
  select count(distinct year) as period FROM bigquery-public-data.chicago_crime.crime;
  ''', project_id='valid-logic-332507', dialect='standard')
df.head()

#answer: the dataset covers a period of 21 years

In [ ]:
#task 2
df = pd.io.gbq.read_gbq('''
  select year, count(*) as crimes FROM bigquery-public-data.chicago_crime.crime
  group by year
  order by count(*) desc;  
''', project_id='valid-logic-332507', dialect='standard')
df.head(period)

#answer: year 2002 had the largest number of crimes

In [ ]:
#task 3, question 1
df = pd.io.gbq.read_gbq('''
  select year, countif(arrest = 1)/count(*) as share from (
  select year, case when arrest= True then 1 else 0 end as arrest, from bigquery-public-data.chicago_crime.crime
  ) group by year
  order by share desc;
  ''', project_id='valid-logic-332507', dialect='standard')
df.head(period)

#answer: year 2005 had the largest share of crimes that led to an arrest

In [ ]:
#task 3, question 2
df = pd.io.gbq.read_gbq('''
  select year, count(*) as crimes FROM bigquery-public-data.chicago_crime.crime
  group by year
  order by year desc;  
''', project_id='valid-logic-332507', dialect='standard')
df.head(period)

#answer: except for the periods 2001-2002 and 2015-2016, for which the trend of crimes had slightly increased, the general trend is of decreasing number of crimes from 2002 until 2021.

In [ ]:
#task 5
df = pd.io.gbq.read_gbq('''
  select year, countif(arrest = 1) as arrests_true from (
  select year, case when arrest= True then 1 else 0 end as arrest, from bigquery-public-data.chicago_crime.crime
  ) group by year
  order by arrests_true desc;
  ''', project_id='valid-logic-332507', dialect='standard')
df.head(period)

#answer: year 2004 had the largest number of crimes that led to an arrest